NOTEBOOK SETUP: IMPORT AND MAP TEST

In [60]:
import pandas as pd
import folium

EECP_map = folium.Map(location=[38.339, -85.674], zoom_start=11)
EECP_map

PLOTTING CRASH DATA

In [61]:
accidents_df = pd.read_excel('accidents.xlsx')

In [62]:
accidents_df['Collision Date'] = pd.to_datetime(accidents_df['Collision Date'])
accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 43 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Master Record Number           237 non-null    int64         
 1   Collision Date                 237 non-null    datetime64[ns]
 2   Hour of Collision Timestamp    237 non-null    object        
 3   County                         237 non-null    object        
 4   City                           237 non-null    object        
 5   Primary Factor                 227 non-null    object        
 6   Manner Of Collision            237 non-null    object        
 7   Secondary Crash Indic          237 non-null    object        
 8   Property Damage Type           2 non-null      object        
 9   Crash Severity Calc.           237 non-null    object        
 10  Injury Nonfatal Number         237 non-null    int64         
 11  Injury Fatal Number

In [63]:
crash_years = sorted(accidents_df['Collision Date'].dt.year.dropna().unique())
crash_years

[2019, 2020, 2021, 2022, 2023, 2024, 2025]

In [64]:
for yr in crash_years:
    crash = folium.FeatureGroup(name=f"Crashes {yr}")

    for _, row in accidents_df[accidents_df['Collision Date'].dt.year == yr].iterrows():
        tooltip_html = (
            f"Collision Date: {row['Collision Date'].date()}<br>"
            f"Collision Time: {row['Hour of Collision Timestamp']}<br>"
            f"Primary Factor: {row['Primary Factor']}<br>"
            f"Manner of Collision: {row['Manner Of Collision']}<br>"
            f"Surface Condition: {row['Surface Condition']}<br>"
            f"Weather: {row['Weather']}"
        )

        folium.CircleMarker(
            location=[row['Latitude Decimal'], row['Longitude Decimal']],
            radius=5,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=1,
            tooltip=folium.Tooltip(tooltip_html, sticky=True, parse_html=True)
        ).add_to(crash)

    crash.add_to(EECP_map)

PLOTTING FRICTION DATA

In [65]:
friction_df = pd.read_excel('friction_data.xlsx')
friction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9270 entries, 0 to 9269
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   type             9270 non-null   object        
 1   route            9270 non-null   object        
 2   direction        9224 non-null   object        
 3   lane             9270 non-null   object        
 4   date             9270 non-null   datetime64[ns]
 5   friction_number  9270 non-null   float64       
 6   latitude         9270 non-null   float64       
 7   longitude        9270 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 579.5+ KB


In [66]:
friction_df['date'] = pd.to_datetime(friction_df['date'])

In [67]:
def colour(fn: float) -> str:
    if fn > 35:
        return 'green'
    if fn >= 25:
        return 'yellow'
    return 'red'

In [75]:
colour(42)

'green'

In [72]:
friction_years = sorted(friction_df['date'].dt.year.dropna().unique())

In [73]:
friction_years

[2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [69]:
for yr in friction_years:
    fg = folium.FeatureGroup(name=f"Friction {yr}", show=False)

    rows = friction_df[friction_df['date'].dt.year == yr]
    for _, r in rows.iterrows():
        tooltip_html = (
            f"Friction Number: {r['friction_number']}<br>"
            f"Date: {r['date'].date()}<br>"
            f"Lane: {r['lane']}"
        )
        folium.CircleMarker(
            location=[r['latitude'], r['longitude']],
            radius=6,
            color=colour(r['friction_number']),
            fill=True,
            fill_color=colour(r['friction_number']),
            fill_opacity=0.9,
            tooltip=folium.Tooltip(tooltip_html, sticky=True, parse_html=True)
        ).add_to(fg)

    fg.add_to(EECP_map)

In [70]:
folium.LayerControl(collapsed=False).add_to(EECP_map)

In [82]:
EECP_map.save('crash_friction_map.html')